In [1]:
d = {"key":"value", "key1":"value1"}

In [2]:
d['key']

'value'

In [3]:
d.key

AttributeError: 'dict' object has no attribute 'key'

In [4]:
from box import ConfigBox

In [5]:
d2 = ConfigBox({"key":"value", "key1":"value1"})

In [6]:
d2

ConfigBox({'key': 'value', 'key1': 'value1'})

In [7]:
d2.key

'value'

In [8]:
def get_prodict(x:int, y:int) -> int:
    return x*y

In [9]:
get_prodict(x=2, y=4)

8

In [10]:
get_prodict(x=2, y="4")

'44'

In [11]:
from ensure import ensure_annotations

In [12]:
@ensure_annotations
def get_prodict(x:int, y:int) -> int:
    return x*y

In [13]:
get_prodict(x=2, y=4)

8

In [14]:
get_prodict(x=2, y="4")

EnsureError: Argument y of type <class 'str'> to <function get_prodict at 0x0000025CC1142310> does not match annotation type <class 'int'>

In [37]:
import torchvision
from torchinfo import summary
# Create EffNetB2 pretrained weights, transforms and model
weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
# transforms = weights.transforms()
model1 = torchvision.models.efficientnet_b2(weights=weights)
model2 = torchvision.models.efficientnet_b2(weights=weights)

In [3]:
for param in model1.parameters():
    param.requires_grad = False

In [4]:
# Get a summary of the model (uncomment for full output)
summary(model1, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   False
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16

In [38]:
for param in list(model2.parameters())[:5]:
    param.requires_grad = False

        

In [39]:
# Get a summary of the model (uncomment for full output)
summary(model2, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   Partial
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32

In [53]:
# Freeze layers in base model
import torch 
from torch import nn
for param in model1.parameters():
    param.requires_grad = False

# Change classifier head with random seed for reproducibility
torch.manual_seed(122)
model1.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=140, out_features=3),
)

In [59]:
list(model2.children())[-3]
type(model1), type(model2)

(torchvision.models.efficientnet.EfficientNet,
 torchvision.models.efficientnet.EfficientNet)

In [75]:
path, dirname

('/home/User/Documents/texte.txt', '/home/User/Documents')

In [6]:
from cnnClassifier.entity.config_entity import TrainingConfig
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models.efficientnet import EfficientNet
from pathlib import Path
from box import ConfigBox
from cnnClassifier import logger
import torch
import os
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from torch import nn
from torch import optim  
import pandas as pd
from torchvision import transforms


train_data_dir = '/home/branis/Downloads/food-vision-MLOps-Projects/artifacts/data_ingestion/data/train'
validation_data_dir = '/home/branis/Downloads/food-vision-MLOps-Projects/artifacts/data_ingestion/data/test'


# Use ImageFolder to create dataset(s)
data_train_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),  # Random horizontal flip
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Perturb colors
            transforms.RandomRotation(10),  # Random rotation by ±10 degrees
            transforms.ToTensor(),  # Convert to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
        ])


data_val_transform = transforms.Compose([
    transforms.RandomRotation(10),  # Random rotation by ±10 degrees
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Get class names
class_names = ['Banis', 'Aida']

# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(train_data_dir, transform=data_train_transform)
val_data = datasets.ImageFolder(validation_data_dir, transform=data_val_transform)


# Turn images into data loaders
train_dataloader = DataLoader(
    train_data,
    batch_size=16,
    shuffle=True,
    num_workers=os.cpu_count(),
    pin_memory=True,
)
val_dataloader = DataLoader(
    val_data,
    batch_size=16,
    shuffle=True,
    num_workers=os.cpu_count(),
    pin_memory=True,
)

dataloader = ConfigBox(
    {
        'train_dataloader': train_dataloader,
        'val_dataloader': val_dataloader,
        'class_names': class_names
    }
)


In [8]:
dataloader.train_dataloader

In [41]:
import random

metrics_df = {
                "Epoch": [], 
                "train_loss": [], 
                "train_acc": [], 
                "val_loss": [], 
                "val_acc:": []
            }
        
for i in range(1):
    for key, val in metrics_df.items():
        val.append(random.randint(1, 20))

In [40]:
df = pd.DataFrame(metrics_df)
last_row = df.iloc[-1]
last_row

Epoch          9
train_loss     1
train_acc      2
val_loss      14
val_acc:      14
Name: 0, dtype: int64

In [45]:
box = ConfigBox(metrics_df)
box.train_acc[-1]


6

In [34]:
model = torch.load('/home/branis/Downloads/food-vision-MLOps-Projects/artifacts/training/last_model.pt')

In [37]:
for key, val in model.items():
    print(f'{key} : {val}')

model_state_dict : OrderedDict([('features.0.0.weight', tensor([[[[ 4.5455e-02,  7.6288e-02,  3.0048e-02],
          [ 5.5432e-01,  2.9662e-01, -3.1593e-02],
          [ 7.8309e-01,  2.3778e-01, -7.0381e-02]],

         [[-2.9542e-01, -2.2617e-01,  4.4444e-02],
          [-1.1007e+00, -7.1409e-01, -5.0454e-02],
          [-1.4469e+00, -6.2652e-01, -6.3332e-03]],

         [[ 2.2284e-01,  1.2584e-01, -2.9721e-02],
          [ 6.2338e-01,  4.4390e-01,  4.5707e-02],
          [ 6.7145e-01,  4.5727e-01,  7.3647e-03]]],


        [[[-3.9284e-02, -2.5299e-02,  1.4641e-02],
          [ 5.2415e-02, -2.0318e-01,  1.3882e-01],
          [ 9.2027e-02, -9.3667e-01,  9.1768e-01]],

         [[-7.7375e-02, -2.6244e-02,  4.9299e-02],
          [ 2.1007e-01, -6.1103e-01,  4.3788e-01],
          [ 6.8553e-02, -1.9628e+00,  1.9089e+00]],

         [[-5.9978e-02,  1.4608e-02,  1.5315e-03],
          [ 1.6183e-01, -1.6718e-01,  9.2858e-02],
          [-6.7604e-03, -4.1440e-01,  3.6333e-01]]],


        [[

In [134]:
import torch 
import torchvision

path = '/home/branis/Downloads/food-vision-MLOps-Projects/artifacts/training/best_model.pt'

# Get model
torch.manual_seed(122)
weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
model = torchvision.models.efficientnet_b2(weights=weights)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1408, out_features=3),

)

# Get optimozer
optimizer = optim.Adam(model.parameters(), lr=0.001) #TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
data_transforms = checkpoint['data_transforms']
loss_fn = checkpoint['loss_fn']


In [136]:
# model.eval()

In [140]:
data_transforms.data_val_transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [139]:
loss_fn

CrossEntropyLoss()

In [141]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)